In [ ]:
#reference link: https://hands-on-tech.github.io/2020/03/15/k8s-jenkins-example.html
#install traefix
helm upgrade --install traefik traefik/traefik -f values.yaml
#portforward traefik pod to localhost
kubectl get pods --selector "app.kubernetes.io/name=traefik" --output=name
#access front-end with address
http://localhost:9000/dashboard/#/
#add kubernetes-dashboard to helm chart local
helm repo add kubernetes-dashboard https://kubernetes.github.io/dashboard/
#install kubernetes-dashboard with default value
helm install kubernetes-dashboard kubernetes-dashboard/kubernetes-dashboard
#port-forward kubernetes-dashboard 
export POD_NAME=$(kubectl get pods -n default -l "app.kubernetes.io/name=kubernetes-dashboard,app.kubernetes.io/instance=kubernetes-dashboard" -o jsonpath="{.items[0].metadata.name}")
echo https://127.0.0.1:8443/
kubectl -n default port-forward $POD_NAME 8443:8443
#install jenkins with helm
chart=jenkinsci/jenkins
helm upgrade --install jenkins -f jenkins-values.yaml $chart
# get jenkins url
jsonpath="{.spec.ports[0].nodePort}"
NODE_PORT=$(kubectl get -n jenkins -o jsonpath=$jsonpath services jenkins)
jsonpath="{.items[0].status.addresses[0].address}"
NODE_IP=$(kubectl get nodes -n jenkins -o jsonpath=$jsonpath)
echo http://$NODE_IP:$NODE_PORT/login

In [ ]:
#install jenkin in kubernetes use helm
#reference link: https://www.jenkins.io/doc/book/installing/kubernetes/
#create namespace jenkins
kubectl create namespace jenkins
#check namespace created or not
kubectl get namespaces
#add chart to helm local
helm repo add jenkinsci https://charts.jenkins.io
#update local helm repo
helm repo update
#add jenkins helm chart
helm repo add jenkinsci https://charts.jenkins.io
helm repo update
#check chart in local repo
helm search repo jenkinsci
#create persistent volume (pv)
kubectl apply -f jenkins-volume.yaml
#get inside minikube container and change persistent jenkins-volume
minikube ssh
sudo chown -R 1000:1000 /data/jenkins-volume
#create service account
kubectl apply -f jenkins-sa.yaml
#config jenkins-values.yaml and then install with helm
chart=jenkinsci/jenkins
helm install jenkins -n jenkins -f jenkins-values.yaml $chart
helm upgrade --install jenkins $chart -n jenkins -f jenkins-values.yaml 
#get admin user password
jsonpath="{.data.jenkins-admin-password}"
secret=$(kubectl get secret -n jenkins jenkins -o jsonpath=$jsonpath)
echo $(echo $secret | base64 --decode)
#get jenkins url
jsonpath="{.spec.ports[0].nodePort}"
NODE_PORT=$(kubectl get -n jenkins -o jsonpath=$jsonpath services jenkins)
jsonpath="{.items[0].status.addresses[0].address}"
NODE_IP=$(kubectl get nodes -n jenkins -o jsonpath=$jsonpath)
echo http://$NODE_IP:$NODE_PORT/login